# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


## 1. Calling the libraries

In [1]:
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from tqdm import tqdm

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from io import StringIO
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from tqdm import tqdm

In [ ]:
#Setting the driver
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()))
driver

In [ ]:
#Setting the page
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get( url )
driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")

In [ ]:
#First menu
elections = driver.find_element( By.XPATH  , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
time.sleep(5)

In [ ]:
#Presidential elections
elections = driver.find_element( By.XPATH  , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()
time.sleep(5)

In [ ]:
#Second menu
elections = driver.find_element(By.XPATH  , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
time.sleep(5)

In [ ]:
#We show the elements of the second menu
elections_list = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')
time.sleep(5)

# Create a list of elements representing election options
options_elections_list = list(elections_list.find_elements(By.CLASS_NAME, 'item'))

# Create a dictionary mapping the text of each option to the element itself
dict_options = { option.text: option for option in options_elections_list }

dict_options

In [ ]:
# Create an empty dataframe for our scrapping
df_results = pd.DataFrame()

# Iterate over the keys of the dictionary, which are the election options
# For this part, we change the waiting time because shorter times have stopped the code from running before
for election_txt in list(dict_options.keys())[1:]:  # Excluding "Seleccionar"
    election = dict_options[election_txt]
    election.click()

    # Wait until the election data button is clickable and click it
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button'))
    ).click()

    # Wait until the "Candidates and results" tab is clickable and click it
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a'))
    ).click()

    # Wait until the table is present
    table = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]'))
    )

    # Read the HTML of the table
    table_html = table.get_attribute('innerHTML')
    table = pd.read_html(table_html)

    # Create a dataframe
    df = table[0]

    # Create a new DataFrame containing only the relevant columns and add the election year
    df = df[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']].copy()
    df.insert(0, 'ELECCIONES', election_txt)

    # Concatenate results in df_results
    df_results = pd.concat([df_results, df], ignore_index=True)

    # Go back to start over
    driver.back()
    time.sleep(15)

    driver.back()
    time.sleep(15)
    
    # Click on the second list to choose the election
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div'))
    ).click()
    time.sleep(15)

In [ ]:
#Show the results
df_results

In [ ]:
# We create the name of the xlsx file
file_name = "df_results.xlsx"

# We place it in the same route
current_dir = os.getcwd()

# We join the route and the file name to create the file path
file_path = os.path.join(current_dir, file_name)

# Saving the file
df_results.to_excel(file_path, index=False)

## 2. Launch/Set the Driver

In [3]:
pwd

'C:\\Users\\Mauricio\\Documentos\\Python\\Diplomado_PUCP\\Lecture_7\\Assignment_5'

In [3]:
# Crea una instancia del Service con el path de chromedriver utilizando webdriver_manager
service = Service(ChromeDriverManager().install())

# Opciones para Chrome
options = Options()
# Puedes agregar opciones a 'options' si es necesario, como options.add_argument("--headless")

# Inicializa el WebDriver con el servicio y las opciones
driver = webdriver.Chrome(service=service, options=options)

In [5]:
# Configuramos el taamaño de la ventana y el zoom
driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")

## 3. Indicating the web page and actions

In [7]:
url = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url)
time.sleep(7)

In [11]:
# We'll click on the first [SELECCIONE], the one for the elections type
#driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/i').click()

driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()

# Next, we choose (click on) ELECCIONES PRESIDENCIALES
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()

# Then, click on the [SELECCIONE] on the right (under Elección)
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
# Now, click on the most recent election
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[2]').click()
# Finally, we click on the red button
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button/span').click()
time.sleep(7)


In [17]:
# In the new page, we'll click on CANDIDATOS Y RESULTADOS
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a').click()
time.sleep(7)

# Once there, we'll get the full table of RESULTADOS ELECTORALES
table_path = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]')
table_html_io = StringIO(table_path.get_attribute('innerHTML'))
table = pd.read_html(table_html_io)
table[0]

,ORGANIZACIÓN POLÍTICA,SÍMBOLO,PLAN DE GOBIERNO,TOTAL VOTOS,PORCENTAJE DE VOTOS VÁLIDOS,LISTA DE CANDIDATOS
0,PARTIDO POLITICO NACIONAL PERU LIBRE,NaN,NaN,8836380,50.126%,VER LISTA
1,FUERZA POPULAR,NaN,NaN,8792117,49.874%,VER LISTA
2,VOTOS EN BLANCO,NaN,NaN,121489,NaN,NaN
3,VOTOS NULOS,NaN,NaN,1106816,NaN,NaN


In [19]:
def scrape_presidential_elections(driver):
    results = []

    # Seleccionar tipo de proceso: Elecciones Presidenciales
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()

    # Obtener lista de elecciones
    elections_dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')
    elections = elections_dropdown.find_elements(By.TAG_NAME, 'div')[1:]

    for election in elections:
        election.click()
        time.sleep(7)
        driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div/button/span').click()
        time.sleep(7)

        while True:
            table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
            table = pd.read_html(StringIO(table_html))[0]
            table['Elecciones'] = election.text
            results.append(table[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS', 'Elecciones']])

            # Verificar si existe una siguiente página
            try:
                next_button = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[4]/div[2]/ul/li[4]/a')
                if "disabled" in next_button.get_attribute("class"):
                    break
                next_button.click()
                time.sleep(7)
            except:
                break

        driver.back()
        time.sleep(7)

    pd.concat(results).to_excel('election_results_simple.xlsx', index=False)
    print("Datos guardados en 'election_results_simple.xlsx'.")

In [29]:
def scrape_presidential_elections(driver):
    results = []
    wait = WebDriverWait(driver, 10)

    # Seleccionar tipo de proceso: Elecciones Presidenciales
    element = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')))
    element.click()

    element = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')))
    element.click()

    # Obtener lista de elecciones
    elections_dropdown = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')))
    elections = elections_dropdown.find_elements(By.TAG_NAME, 'div')[1:]

    for election in elections:
        election.click()

        element = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div/button/span')))
        element.click()

        while True:
            table_html = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]'))).get_attribute('innerHTML')
            table = pd.read_html(StringIO(table_html))[0]
            table['Elecciones'] = election.text
            results.append(table[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS', 'Elecciones']])

            # Verificar si existe una siguiente página
            try:
                next_button = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[4]/div[2]/ul/li[4]/a')))
                if "disabled" in next_button.get_attribute("class"):
                    break
                next_button.click()
            except:
                break

        driver.back()

    pd.concat(results).to_excel('election_results_simple.xlsx', index=False)
    print("Datos guardados en 'election_results_simple.xlsx'.")

In [35]:
driver.get(url)

# Ejecutar la función de scraping
scrape_presidential_elections(driver)

TimeoutException: Message: 
